In [36]:
import gpc
import math
import itertools
import re

In [27]:
def Ei(i):
    i = int(i)
    ei = [0,0,0]
    ei[i] = str(i)
    ei[(i+1)%3] = str((2+i)%3)
    ei[(i+2)%3] = str((1+i)%3)
    return ei

In [28]:
def isEipal(seq, i):
    """Checks if a string seq is an E_i palindrome."""
    ei = Ei(i)
    l = len(seq)
    if l == 1:
        if seq == str(i):
            return True
        else:
            return False
    for x in range(0, math.ceil(l/2)):
        if seq[x] != ei[int(seq[l-1-x])]:
            return False
    return(True)

In [43]:
def testPalindromicity(seq):
    """Checks if a seq is an palindrome or and E-palindrome and 
    returns its nature."""
    if isEipal(seq,0):
        return [True, "0"]
    elif isEipal(seq, 1):
        return [True, "1"]
    elif isEipal(seq, 2):
        return [True, "2"]
    elif gpc.isPal(seq):
        return [True, "R"]
    else:
        return [False]

In [44]:
def makeEipalClosure (seq, i):
    """Makes E_i-th palindromic closure of a string."""
    ei = Ei(i)
    if isEipal(seq, i) == True:
        return(seq)
    j = 1
    while isEipal(seq[j:], i) != True:
        j = j+1
    #print("    {0} longest palindromic suffix : {1}".format(seq,seq[j:]))
    closure = seq
    pref = seq[j-1::-1]
    for letter in pref:
        closure = closure + ei[int(letter)]
    return(closure)

In [45]:
def make012Word(delta, theta, steps, seed = ""):
    """Makes a GPS over {0,1,2} from sequences delta and theta."""
    w = seed
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = zpu.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        #print("w{0} = {1}".format(step+1,w))
    return(w)

In [89]:
def is012NormalizedNaive(delta, theta, steps):
    """Checks if delta and theta are normalized and if not, 
    returns the beginning of the normalized sequence."""
    w = ""
    l=1
    prefixes = []
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = gpc.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        prefixes.append(w)
    print(prefixes)
    print(w)
    newdelta = delta[0]
    newtheta = ""
    while l <= len(w):
        prefix = w[:l]
        res = testPalindromicity(prefix)
        if res[0] == True:
            if l< len(w):
                newdelta = newdelta + w[l]
            newtheta = newtheta + res[1]           
        l=l+1
    if newdelta == delta[:steps] and newtheta == theta[:steps]:
        return [True]
    else:
        return [False, newdelta, newtheta]
    
    # The length of newdelta and newtheta are the same since the whole word 
    # is an palindrome because if was generated by the GPS construction
    # from delta and theta.

## Implementation of the normalization algorithm

### Preprocessing of the normalized bi-sequence for our implementation

In our implementation of the algorithm, we decided to work only with words
that has 0 as first letter, 1 at second and 2 at third in order to make the 
algorithm easier to read and write. Of course, we want it working for all
the bi-sequences, so we have to preprocess the bi-sequence so that the word
obtained has first 0, then 1 and then 2. After the result of the algorithm,
we will go back to the original letters order.

### Preprocessing of the beginning of the bi-sequence

We want that every word beginning with $i^l$ (where $l$ is the largest
possible) has a bi-directive sequence $(\Delta, \Theta)$ where the prefix
of $\Theta$ of lenght $l$ is equal to $E^l_i$. (We only solve the cases
when there is an $R$ instead of $E_0$ (e.g. $(0000, RE_0RE_0) \to (0000, E_0E_0E_0E_0)$, since the other cases are then solved
within the normalization process.

In [190]:
def beginningNormalization(delta, theta):
    biseq = gpc.makeBiseq(delta, theta)
    m = re.match("(0(R|0))+", biseq)
    if m:
        biseq = "00"*int((m.end()-m.start())/2) + biseq[m.end():]
    return gpc.parseBiseq(biseq)

In [191]:
beginningNormalization("000011", "R0R021")

['000011', '000021']

### Auxiliary functions

<img src="prefixesrules.jpg",width=600>
    
    
    

In [198]:
# List: bad prefix regex --> the new two symbols instead of the last one
bad_prefixes = [
    ["(00)*02", "0012"], # 1.
    ["0012(0R12)*0R10", "1220"], # 2.
    ["0012(0R12)+01", "0R21"],  
]

### Normalization algorithm

In [15]:
print(re.search('01$', '010101'))

<_sre.SRE_Match object; span=(4, 6), match='01'>


In [ ]:
def normalize(delta, theta):
    """Returns the normalized directive bi-sequence giving the same GPS word
    as (delta, theta)"""
    # Normalization of the letters order
    
    
    # Normalization of the prefix
    [delta, theta] = beginningNormalization(delta, theta)
    
    biseq = makeBiseq(delta, theta)
    bad_matches = find_bad_factors()
    
    # The main algorithm

In [ ]:
def is012Normalized(delta, theta, steps):
    "kontroluje jestli muzou byt delta a theta normalizovane"
    w = ""
    l=1
    prefixes = []
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = zpu.makePalClosure(w)
        elif theta[step] in ["0", "1", "2"]:
            w = makeEipalClosure(w, theta[step])
        else:
            print("wrong symbol")
            break
        prefixes.append(w)
    #print(prefixes)
    #print(w)
    while l <= len(w):
        prefix = w[:l]
        if zpu.isPal(prefix) or isEipal(prefix,0) or isEipal(prefix,0) or isEipal(prefix,0):
            if prefix in prefixes:
                #print(prefix + ": OK")
                pass
            else:
                #print("prefixy neobsahuji: " + prefix)
                return False
        l=l+1
    return True

is012Normalized("0100001","R012RR2", 7)

In [16]:
isEipal("012", 1)

True

In [17]:
a = [0,1, "ahoj"]
a = [a, "dalsi"]